In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install accelerate==0.20.3
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install tensorboard --quiet

In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
MODEL_NAME = "tiiuae/falcon-40b-instruct"
#MODEL_NAME = "togethercomputer/RedPajama-INCITE-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
!pip install datasets
from datasets import load_dataset
data = load_dataset("Weni/LLM-base", split='train')
df = data.to_pandas()
df = selected_rows_df.sample(frac=1)
df.reset_index(drop=True, inplace=True)

def categorise(row):
  return "Responda à pergunta com a maior sinceridade possível usando o CONTEXTO fornecido e, se a resposta não estiver contida no CONTEXTO abaixo, diga 'Desculpe, não possuo essa informação'. \n\nCONTEXTO: {} \nPergunta: {} \nResposta: {}".format(row['context'],row['question'],row['resposta'])


from datasets import Dataset, DatasetDict

# df = pd.read_csv('dados _limpos_final.csv')

df['prompt'] = df.apply(lambda x: categorise(x), axis=1)
del df['id']
del df['index']
del df['correct_ans']
del df['question']
del df['resposta']
del df['context']

data = DatasetDict()
data['train'] = Dataset.from_pandas(df)

def generate_and_tokenize_prompt(data_point):
    tokenized_full_prompt = tokenizer(data_point["prompt"], padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle().map(generate_and_tokenize_prompt)
data

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
OUTPUT_DIR = "experiments"
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-5,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=1200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

In [ ]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")